# License Plate Character Recognition Training with YOLOv8

Notebook này huấn luyện model YOLOv8 để nhận diện các ký tự trên biển số xe

In [ ]:
# Cài đặt thư viện cần thiết
!pip install ultralytics

from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import yaml
import os

# Kiểm tra GPU
print(f"Using CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name()}")

In [ ]:
# Tạo model YOLOv8n mới
model = YOLO('yolov8n.pt')

# Train với cấu hình tối ưu cho nhận dạng ký tự
results = model.train(
    data='Letter_detect.yaml',
    epochs=100,
    imgsz=640,
    batch=32,
    device=0 if torch.cuda.is_available() else 'cpu',
    name='LP_ocr_v8',
    # Data augmentation
    hsv_h=0.015,  # Ít thay đổi màu sắc
    hsv_s=0.3,
    hsv_v=0.2,
    degrees=2.0,   # Xoay nhẹ để không làm biến dạng ký tự
    translate=0.1,
    scale=0.2,
    shear=0.0,    # Không shear để tránh biến dạng
    perspective=0.0,  # Không perspective
    flipud=0.01,   # Hạn chế lật ảnh
    fliplr=0.01,
    mosaic=0.8,
    mixup=0.1,
    copy_paste=0.1,
    # Tham số huấn luyện
    warmup_epochs=3,
    patience=50,
    lr0=0.01,
    lrf=0.001,
    momentum=0.937,
    weight_decay=0.0005,
    # Tối ưu cho OCR
    box=7.5,      # Box loss gain
    cls=0.3,      # Class loss gain
    dfl=1.5,      # DFL loss gain
    close_mosaic=10  # Tắt mosaic trong 10 epoch cuối
)

## Vẽ Training Metrics
Đánh giá quá trình huấn luyện qua các metrics

In [ ]:
plt.figure(figsize=(20, 10))

# Plot loss
plt.subplot(2, 3, 1)
plt.plot(results.results_dict['train/box_loss'], label='train')
plt.plot(results.results_dict['val/box_loss'], label='val')
plt.title('Box Loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(2, 3, 2)
plt.plot(results.results_dict['train/cls_loss'], label='train')
plt.plot(results.results_dict['val/cls_loss'], label='val')
plt.title('Classification Loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(2, 3, 3)
plt.plot(results.results_dict['train/dfl_loss'], label='train')
plt.plot(results.results_dict['val/dfl_loss'], label='val')
plt.title('DFL Loss')
plt.xlabel('Epoch')
plt.legend()

# Plot metrics
plt.subplot(2, 3, 4)
plt.plot(results.results_dict['metrics/precision'], label='precision')
plt.plot(results.results_dict['metrics/recall'], label='recall')
plt.title('Precision & Recall')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(2, 3, 5)
plt.plot(results.results_dict['metrics/mAP50(B)'], label='mAP50')
plt.plot(results.results_dict['metrics/mAP50-95(B)'], label='mAP50-95')
plt.title('Mean Average Precision')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(2, 3, 6)
if 'metrics/fitness' in results.results_dict:
    plt.plot(results.results_dict['metrics/fitness'])
    plt.title('Model Fitness')
    plt.xlabel('Epoch')

plt.tight_layout()
plt.show()

## Kiểm tra Model
Test model trên một số ảnh biển số

In [ ]:
# Load best model
best_model = YOLO('runs/detect/LP_ocr_v8/weights/best.pt')

# Test trên một số ảnh biển số
test_images = ['../test_image/bien_so.jpg']

for img_path in test_images:
    # Predict với confidence threshold thấp hơn để detect được nhiều ký tự
    results = best_model.predict(
        source=img_path,
        conf=0.25,
        iou=0.45,
        agnostic_nms=True,
        max_det=100,  # Số lượng detection tối đa
        save=True
    )
    
    # Hiển thị kết quả
    plt.figure(figsize=(12, 8))
    plt.imshow(results[0].plot())
    plt.axis('off')
    plt.title('Character Detection Results')
    plt.show()
    
    # In ra các ký tự được detect
    print("\nDetected characters:")
    boxes = results[0].boxes
    chars = []
    for i in range(len(boxes)):
        cls = int(boxes.cls[i].item())
        conf = float(boxes.conf[i].item())
        x1 = float(boxes.xyxy[i][0].item())
        chars.append((x1, cls))
    
    # Sắp xếp ký tự từ trái sang phải
    chars.sort(key=lambda x: x[0])
    plate_number = ''.join([cfg['names'][c[1]] for c in chars])
    print(f"License plate number: {plate_number}")

## Export Model
Xuất model sang các định dạng khác nhau để triển khai

In [ ]:
# Kiểm tra kết quả trên một số ảnh biển số
best_model_path = 'runs/detect/LP_OCR_yolov8_new/weights/best.pt'
model = YOLO(best_model_path)

# Sử dụng ảnh test từ thư mục test_images
test_images = ['../test_image/bien_so.jpg']

for img_path in test_images:
    results = model.predict(img_path, conf=0.25)
    
    # Hiển thị kết quả
    plt.figure(figsize=(10, 4))
    plt.imshow(results[0].plot())
    plt.axis('off')
    plt.show()